In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer, confusion_matrix
import matplotlib.pyplot as plt
from sklearn import linear_model
import xgboost as xgb
from functools import reduce
%matplotlib inline